In [1]:
import pandas as pd
import os
import glob 
from datetime import datetime


##### Generate the columns for merge of footystats and football odds 

In [2]:
""" 
The following columns are not required for analysis.
Hence, they are dropped.
"""
cols= ['timestamp', 'date_GMT', 'status', 'attendance', 
       'referee', 'GameWeek', 'Pre-MatchPPG(Home)', 'Pre-MatchPPG(Away)',
       'home_ppg', 'away_ppg',  
       'home_team_goal_timings', 'away_team_goal_timings',
       'home_team_first_half_cards', 'home_team_second_half_cards',
       'away_team_first_half_cards', 'away_team_second_half_cards',
       'home_team_shots_off_target',
       'away_team_shots_off_target',
       'home_team_possession', 'away_team_possession', 'team_a_xg',
       'team_b_xg', 'average_goals_per_match_pre_match',
       'btts_percentage_pre_match', 'over_15_percentage_pre_match',
       'over_25_percentage_pre_match', 'over_35_percentage_pre_match',
       'over_45_percentage_pre_match', 'over_15_HT_FHG_percentage_pre_match',
       'over_05_HT_FHG_percentage_pre_match',
       'over_15_2HG_percentage_pre_match', 'over_05_2HG_percentage_pre_match',
       'average_corners_per_match_pre_match',
       'average_cards_per_match_pre_match', 'odds_ft_home_team_win',
       'odds_ft_draw', 'odds_ft_away_team_win', 'odds_ft_over15',
       'odds_ft_over25', 'odds_ft_over35', 'odds_ft_over45', 'odds_btts_yes',
       'odds_btts_no', 'stadium_name','Country', 'League', 'Season',
       'Time', 'HG', 'AG', 'Res', 'MaxH', 'MaxD', 'MaxA']

""" Renaming all the columns as per ML modelling featues"""

dict_merge ={'Home': 'HomeTeam',
       'Away': 'AwayTeam',
       'home_team_goal_count': 'FTHG',
       'away_team_goal_count': 'FTAG',
       'home_team_goal_count_half_time': 'HTHG', 
       'away_team_goal_count_half_time': 'HTAG',
       'home_team_shots': 'HS',
       'away_team_shots': 'AS',
       'home_team_shots_on_target': 'HST',
       'away_team_shots_on_target': 'AST',
       'home_team_fouls': 'HF',
       'away_team_fouls': 'AF',
       'home_team_corner_count': 'HC', 
       'away_team_corner_count': 'AC',
       'home_team_yellow_cards': 'HY',
       'home_team_red_cards': 'AY',
       'away_team_yellow_cards': 'HR',
       'away_team_red_cards': 'AR',
       'PH': 'PSH', 
       'PD': 'PSD',
       'PA': 'PSA',
       'AvgH': 'B365H',
       'AvgD': 'B365D',
       'AvgA': 'B365A'}

##### Generate the columns for generating fixtures from **footystats** 

In [3]:
""" these are the fixture columns dropped"""

cols_fix = ['timestamp', 'date_GMT', 'status', 'attendance',
       'referee', 'GameWeek', 'Pre-MatchPPG(Home)', 'Pre-MatchPPG(Away)',
       'home_ppg', 'away_ppg', 'home_team_goal_count', 'away_team_goal_count',
       'total_goal_count', 'total_goals_at_half_time',
       'home_team_goal_count_half_time', 'away_team_goal_count_half_time',
       'home_team_goal_timings', 'away_team_goal_timings',
       'home_team_corner_count', 'away_team_corner_count',
       'home_team_yellow_cards', 'home_team_red_cards',
       'away_team_yellow_cards', 'away_team_red_cards',
       'home_team_first_half_cards', 'home_team_second_half_cards',
       'away_team_first_half_cards', 'away_team_second_half_cards',
       'home_team_shots', 'away_team_shots', 'home_team_shots_on_target',
       'away_team_shots_on_target', 'home_team_shots_off_target',
       'away_team_shots_off_target', 'home_team_fouls', 'away_team_fouls',
       'home_team_possession', 'away_team_possession', 'team_a_xg',
       'team_b_xg', 'average_goals_per_match_pre_match',
       'btts_percentage_pre_match', 'over_15_percentage_pre_match',
       'over_25_percentage_pre_match', 'over_35_percentage_pre_match',
       'over_45_percentage_pre_match', 'over_15_HT_FHG_percentage_pre_match',
       'over_05_HT_FHG_percentage_pre_match',
       'over_15_2HG_percentage_pre_match', 'over_05_2HG_percentage_pre_match',
       'average_corners_per_match_pre_match',
       'average_cards_per_match_pre_match', 'odds_ft_over15',
       'odds_ft_over25', 'odds_ft_over35', 'odds_ft_over45', 'odds_btts_yes',
       'odds_btts_no', 'stadium_name']

""" renaming the columns for generating fixtures"""
dict_fix ={'Home': 'HomeTeam',
       'Away': 'AwayTeam',
       'odds_ft_home_team_win': 'PSH',
       'odds_ft_draw': 'PSD',
       'odds_ft_away_team_win': 'PSA'}

##### Get all the files in the path (Argentina, Japan, Ireland, Brazil , USA)


In [4]:
#os.chdir('src/merge_footystats/')
PATH = os.getcwd()

def datefromtimestamp(timestamp):
    date = datetime.utcfromtimestamp(timestamp).strftime('%Y-%m-%d')
    return date
    
class FootyStats:
    def __init__(self):
        pass

    @staticmethod
    def merge_stats_odds(_df1:pd.DataFrame, _df2:pd.DataFrame)-> pd.DataFrame:
        df1 = _df1
        df2 = _df2
        _lst = [datefromtimestamp(timestamp) for timestamp in df1['timestamp']]
        df1['Date'] = _lst
        df1 = df1.assign(Date = lambda df1:pd.to_datetime(df1.Date))

        format_str = '%d/%m/%Y'
        _df2_lst = [(datetime.strptime(date,format_str)) for date in df2['Date']]
        _date_lst = [item.strftime('%Y/%m/%d') for item in _df2_lst]
        df2['Date'] = _date_lst
        df2 = df2.assign(Date = lambda df2:pd.to_datetime(df2.Date))
        
        """ format both the dataframes before merging"""
        df1 = df1[df1.status == 'complete']
        df1.rename(columns = {'home_team_name':'Home', 'away_team_name':'Away'}, inplace = True)

        """ removing whitespaces"""
        df1.columns = df1.columns.str.replace(' ', '')
        df2.columns = df2.columns.str.replace(' ', '')

        """merge both the dataframes"""
        df = pd.merge(df1, df2,on=['Date','Home','Away'], how = 'inner')

        """ dropping all the un-required cols"""
        df.drop(cols, axis=1, inplace= True)
        df.rename(columns=dict_merge,
          inplace=True)
        return df

    @staticmethod
    def get_fixtures(df1:pd.DataFrame)-> pd.DataFrame:
      """ generating fixtures from footystats data"""
      df3 = df1
      df3 = df3[df3.status == 'incomplete']
      _lst = ([ datefromtimestamp(timestamp) for timestamp in df3.timestamp])
      df3['Date'] = _lst
      df3 = df3.assign(Date = lambda df3:pd.to_datetime(df3.Date))
      
      df3.rename(columns = {'home_team_name':'Home', 'away_team_name':'Away'}, inplace = True)
      df3.columns = df3.columns.str.replace(' ', '')
      df3.drop(cols_fix, axis=1, inplace=True)
      df3.rename(columns=dict_fix,
                inplace=True)

      return df3  

In [5]:
"""
Get all the merged files generated  from footystats and football data co uk
"""
# use glob to get all the csv files 
# in the folder
base_path = os.getcwd()
path_footystats = base_path+('/merge_data/footystats_data/')
csv_files_footystats = glob.glob(os.path.join(path_footystats, "*.csv"))
dict_odds = {}
path_odds = base_path+('/merge_data/football_odds_data/')
csv_files_odds = glob.glob(os.path.join(path_odds, "*.csv"))

In [6]:
"""
Get all the merged files generated  from footystats and football data co uk
"""
# use glob to get all the csv files 
# in the folder
base_path = os.getcwd()
#path_footystats = base_path+('/merge_data/footystats_data/')
#csv_files_footystats = glob.glob(os.path.join(path_footystats, "*.csv"))
csv_files_footystats = ['/Users/nikhilshukla/Desktop/football-data/src/merge_footystats/merge_data/footystats_data/argentina-primera-division-matches-2022-to-2022-stats.csv',
'/Users/nikhilshukla/Desktop/football-data/src/merge_footystats/merge_data/footystats_data/brazil-serie-a-matches-2022-to-2022-stats.csv',
'/Users/nikhilshukla/Desktop/football-data/src/merge_footystats/merge_data/footystats_data/japan-j1-league-matches-2022-to-2022-stats.csv',
'/Users/nikhilshukla/Desktop/football-data/src/merge_footystats/merge_data/footystats_data/usa-mls-matches-2022-to-2022-stats.csv'
 ]
dict_odds = {}
#path_odds = base_path+('/merge_data/football_odds_data/')
#csv_files_odds = glob.glob(os.path.join(path_odds, "*.csv"))
csv_files_odds = [
 '/Users/nikhilshukla/Desktop/football-data/src/merge_footystats/merge_data/football_odds_data/ARG.csv',
 '/Users/nikhilshukla/Desktop/football-data/src/merge_footystats/merge_data/football_odds_data/BRA.csv',
 '/Users/nikhilshukla/Desktop/football-data/src/merge_footystats/merge_data/football_odds_data/USA.csv',
 '/Users/nikhilshukla/Desktop/football-data/src/merge_footystats/merge_data/football_odds_data/JAP.csv'
 ]
dict_odds = {file.split('/')[-1].split('.')[0]:file for file in csv_files_odds}


In [7]:
for f in csv_files_footystats:
    df_footystats = pd.read_csv(f)
    country = f.split('/')[-1].split('-')[0].upper()[:3]
    odd_path = dict_odds.get(country)
    df_odds = pd.read_csv(odd_path)
    df_merge = FootyStats.merge_stats_odds(df_footystats, df_odds)
    df_merge.to_csv(base_path+('/merge_data/final_merged_data/{}_2022.csv'.format(country)))

### Getting Fixtures

In [8]:
"""
Get all the fixtures from footystats week
"""
# use glob to get all the csv files 
# in the folder
base_path = os.getcwd()
path_footystats = base_path+('/merge_data/footystats_data/')
csv_files_footystats = glob.glob(os.path.join(path_footystats, "*.csv"))

for f in csv_files_footystats:
    df_footystats = pd.read_csv(f)
    country = f.split('/')[-1].split('-')[0].upper()[:3]
    division = f.split('/')[-1].split('-')[1].upper()[:3]
    df_fixtures = FootyStats.get_fixtures(df_footystats)
    df_fixtures.to_csv(base_path+('/merge_data/fixtures/fixtures_{}_{}.csv'.format(country,division)))


/var/folders/c4/r5d56mnd3mqd570blq2lrl6m0000gn/T/ipykernel_49357/3679105138.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['Date'] = _lst


In [9]:
"""
Merging all the fixtures into one 
"""
base_path = os.getcwd()
path_fixtures = base_path+('/merge_data/fixtures/')
csv_files_fixtures = glob.glob(os.path.join(path_fixtures, "*.csv"))

fixtures = [ pd.read_csv(file) for file in csv_files_fixtures ]
result = pd.concat(fixtures)
final_fixtures = result.sort_values(['Date'], ascending=True)
final_fixtures = final_fixtures[final_fixtures.PSH != 0]
for f in csv_files_fixtures:
    os.remove(f)
final_fixtures.to_csv(path_fixtures+('/fixtures.csv'))

In [10]:
final_fixtures.drop(['Unnamed: 0'],axis=1,inplace=True)

In [11]:
#final_fixtures.drop(['Unnamed: 0.1'],axis=1,inplace=True)

In [12]:
final_fixtures.to_csv(path_fixtures+('/fixtures.csv'))

### Merge into Allleagues

In [13]:
#if you get the invalid ssh url certificate else move on to the next line of code
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [14]:
import datetime
#run only when fixtures are from football data co uk
def convert_datetime(df2):    
    lst = []
    for i in range(len(df2)):
        lst.append(datetime.datetime.strptime(df2['Date'].iloc[i], "%d/%m/%Y").strftime("%Y-%m-%d")) 
    new_date = pd.Series(lst)
    df2['new_date'] = new_date

    # need to drop the old Date to take in the new updated date series
    df2.drop('Date',axis =1)
    df2 = df2.assign(Date = lambda df: pd.to_datetime(df2.new_date))
    df2['Date'] = df2['Date'].dt.date
    return df2

In [15]:
#get allleagues data
url = 'https://raw.githubusercontent.com/Ericvolten1/football-data/main/src/all_leagues.csv'
allleagues = pd.read_csv(url)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3251: DtypeWarning: Columns (8,124) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [16]:
PATH = os.getcwd()

In [17]:
belgium_path = PATH + '/merge_data/football_odds_data/B1.csv'
bundesliga_path = PATH + '/merge_data/football_odds_data/D1.csv'
bundesliga2_path = PATH + '/merge_data/football_odds_data/D2.csv'
epl_path = PATH + '/merge_data/football_odds_data/E0.csv'
eng_championship_path = PATH + '/merge_data/football_odds_data/E1.csv'
eng_leagueone_path = PATH + '/merge_data/football_odds_data/E2.csv'
ligue1_path = PATH + '/merge_data/football_odds_data/F1.csv'
ligue2_path = PATH + '/merge_data/football_odds_data/F2.csv'
erdevisie_path = PATH + '/merge_data/football_odds_data/N1.csv'
seriea_path = PATH + '/merge_data/football_odds_data/I1.csv'
serieb_path = PATH + '/merge_data/football_odds_data/I2.csv'
laliga_path = PATH + '/merge_data/football_odds_data/SP1.csv'
laliga2_path = PATH + '/merge_data/football_odds_data/SP2.csv'
portugal_path = PATH + '/merge_data/football_odds_data/P1.csv'
turkey_path = PATH + '/merge_data/football_odds_data/T1.csv'
scot_premiership_path = PATH + '/merge_data/football_odds_data/SC0.csv'
scot_championship_path = PATH + '/merge_data/football_odds_data/SC1.csv'

In [18]:
belgium = pd.read_csv(belgium_path)
bundesliga = pd.read_csv(bundesliga_path)
bundesliga2 = pd.read_csv(bundesliga2_path)
epl = pd.read_csv(epl_path)
eng_championship = pd.read_csv(eng_championship_path)
eng_leagueone = pd.read_csv(eng_leagueone_path)
ligue1 = pd.read_csv(ligue1_path)
ligue2 = pd.read_csv(ligue2_path)
erdevisie = pd.read_csv(erdevisie_path)
seriea = pd.read_csv(seriea_path)
serieb = pd.read_csv(serieb_path)
laliga = pd.read_csv(laliga_path)
laliga2 = pd.read_csv(laliga2_path)
portugal = pd.read_csv(portugal_path)
turkey = pd.read_csv(turkey_path)
scot_premiership = pd.read_csv(scot_premiership_path)
scot_championship = pd.read_csv(scot_championship_path)


frames = [belgium,bundesliga,bundesliga2, epl,eng_championship,eng_leagueone,
         ligue1,ligue2,erdevisie,seriea,serieb,laliga,laliga2,
         portugal,turkey,scot_premiership,scot_championship]
updated = [convert_datetime(val) for val in frames]

In [19]:
final = pd.concat(updated).drop_duplicates().reset_index(drop=True)

In [20]:
latest_allleagues = pd.concat([allleagues,final]).drop_duplicates().reset_index(drop=True)
latest_allleagues.drop(columns=['Unnamed: 0','new_date'],axis =1,inplace=True)

In [21]:
allleagues_path = PATH + ('/../all_leagues.csv')
latest_allleagues.to_csv(allleagues_path)

### merging allleagues with footystats data

In [22]:
allleagues_path = PATH + '/../all_leagues.csv'
new_allleagues = pd.read_csv(allleagues_path)

In [23]:
merged_path = PATH + '/merge_data/final_merged_data/'
csv_files_merged = glob.glob(os.path.join(merged_path, "*.csv"))
new_frames = [pd.read_csv(file) for file in csv_files_merged]


In [24]:
updated_frames = pd.concat(new_frames).drop_duplicates().reset_index(drop=True)

In [25]:
final_alleagues = pd.concat([new_allleagues,updated_frames]).drop_duplicates().reset_index(drop=True)

In [26]:
allleagues_path = PATH + ('/../all_leagues.csv')
final_alleagues.to_csv(allleagues_path)